<img src="https://bioinf.nl/~davelangers/hanze.png" align="right" />

# <span id="0">Casus *Hidden Markov Model* - Deel III</span>

Inhoud:

* **<a href="#1">Het Viterbi algoritme</a>**

* **<a href="#2">Het decoding probleem</a>**

* **<a href="#3">Rekenvoorbeeld</a>**

* **<a href="#4">Je eigen `HiddenMarkovModel` class</a>**

* **<a href="#5">CpG-eilandjes</a>**

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np

<a id="1" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Het Viterbi algoritme

Het Viterbi-algoritme is een [dynamisch](https://en.wikipedia.org/wiki/Dynamic_programming) programmeeralgoritme voor het verkrijgen van de meest waarschijnlijke reeks verborgen toestanden die resulteert in een reeks waargenomen gebeurtenissen. Dit gebeurt vooral in de context van Hidden Markov modellen, maar er zijn ook andere toepassingen.

Bekijk onderstaande inleidende video waarin het algoritme wordt gebruikt om de kortste route tussen twee bestemmingen te bepalen. Het voordeel van het Viterby-algoritme is dat je niet alle mogelijke routes hoeft op te sommen; tijdens de toepassing van het algoritme vallen vele mogelijk routes die niet optimaal kunnen zijn vanzelf al af, en deze hoef je daardoor niet helemaal door te rekenen.

In [ ]:
%%html
<iframe width="640" height="360" src="https://www.youtube.com/embed/6JVqutwtzmo" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In het onderstaande kaartje staan afstanden (in km) aangegeven om in zes (verschillend gekleurde) stappen van Groningen naar zuid-Nederland te reizen. Gebruik het Viterbi-algoritme om de kortste afstand vanuit Groningen naar elke stad in dit kaartje te bepalen, vergelijkbaar met wat er in de video gedaan wordt. Je mag alleen de richting van de pijlen volgen. Geef als eindresultaat de afstanden van Groningen tot Middelburg, Breda, Tilburg, Eindhoven en Maastricht, plus de paden die je volgt om vanuit Groningen optimaal in die steden terecht te komen.

![Casus_HiddenMarkovModel_Deel3_kaartje.jpg](attachment:Casus_HiddenMarkovModel_Deel3_kaartje.jpg)

[Image credit](https://depositphotos.com/nl/vector/political-map-netherlands-national-borders-cities-rivers-646536752.html)

## Laag 1 
gronigen > leeuwarden = 65

groningen > assen = 35

groningen Emmen = 60

## Laag 2
alkemaar = 65 + 110 = 175

leylestda = 65 + 100= 165
leylestda = 35 + 115= 150!

zwolle = 35 + 75= 110!
zwolle = 60 + 75 = 135

enschde = 60 + 95 = 155

### Laag 3
haarlem= 175 +40 = 215

Amsterdam= 175+ 45 = 220 

Alemer = 150 +  30 = 180

Appledoorn = 150 + 60 = 210
Appeldoorn = 110 + 40 = 150!
appeldoorn = 155+ 75 = 230

Arhem = 155+ 95 = 250

## Laag 4
Den haag = 215 + 60 = 275

leiden = 215+ 45 = 260!
Leiden = 220 + 50 = 270

Utrecht= 220+ 50 = 270 
utecht =180+45 = 225!

Amersfoort= 150 +45= 195!
amersfoort= 250+55= 305

nijmegen = 250+20= 270

## Laag 5 
Rotterdam =275 + 25= 300
Rotterdam = 260 +40 = 300
Rotterdam = 225 +60= 285!

Bosh= 225+ 55= 280
bosh= 195+ 70= 265!
boah = 270 +50= 320

Venlo = 270 +70 = 340

## Laag 6
Middelburg= 285 +120= 405 ook wel Middelburg Rotterdam Utrecht Almere Leylstad assen en dan gronigen

Breda = 285 +50= 335 Breda Rotterdam Utrecht Almere Leylstad assen en dan gronigen

Tilburg = 285+80= 365
Tilburg= 265+ 30= 295 Tilburg bosh Amersfoort Appeldoorn zwolle assen gronigen

Eindhoven = 265+ 40 = 305 eindhoven bosh amersfoort appeldoorn zwolle assen groningen 
Eindhoven = 340 +60= 400

Maastricht= 340+ 75= 415 Maastricht venlo Nijmegen arhem enschde emmen groningen

# EindResultaat
Groningen Assen Leylstad Alemer Utrecht Rotterdam *Middelburg*

Groningen Assen Leylstad Alemer Utrecht Rotterdam *Breda*

Groningen Assen Zwolle Appeldoorn Amersfoort 's Hertogenbosh *Tilburg*

Groningen Assen Zwolle Appeldoorn Amersfoort 's Hertogenbosh *Eindhoven*

Groningen Emmen Enschede Arhem Nijmegen Venlo *Maastricht*


<a id="2" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Het decoding probleem

In een vorige les heb je een reeks knikkers getrokken aan drie verschillende tafels. De gegevens omtrent overgangswaarschijnlijkheden en emissiekansen kun je aflezen uit de inmiddels bekende gegevens omtrent de opzet van het experiment:

| Tafel: |  ❶  |  ❷  |  ❸  |
| -----: | :-: | :-: | :-: |
| **Grabbelton:** | 6x blauw | 2x blauw | 1x blauw |
|                 | 3x geel  | 6x geel  | 0x geel  | 
|                 | 1x groen | 2x groen | 6x groen |
|                 | 2x rood  | 2x rood  | 5x rood  |
| **Dobbelsteen:** | ⚀→① | ⚀→① | ⚀→① |
|                  | ⚁→② | ⚁→② | ⚁→① |
|                  | ⚂→② | ⚂→② | ⚂→① |
|                  | ⚃→② | ⚃→③ | ⚃→① |
|                  | ⚄→③ | ⚄→③ | ⚄→② |
|                  | ⚅→③ | ⚅→③ | ⚅→③ |

In de praktijk van Hidden Markov Models hebben we waarnemingen (kleuren knikkers) maar kennen we niet de toestanden (de tafels). Laten we eens proberen te berekenen wat de *meest waarschijnlijke* reeks tafels is die ten grondslag ligt aan de volgende reeks gekleurde knikkers:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
| **Kleur:** | geel  | groen | blauw | rood  | groen |

We noemen dit het *decoding-probleem*. Hiervoor kunnen we het Viterby-algoritme gebruiken. In dit geval *minimaliseren* we niet de totale *afstand* tussen steden, maar *maximaliseren* we de *kans* op emissies en toestanden. In plaats van bij te houden via welke *steden* je zou moeten reizen houden we bij via welke *toestanden* de uitkomsten het meest waarschijnlijk zijn.

Het Viterbi-algoritme begint met alleen het begin van de reeks te beschouwen: eerst kijken we alleen naar beurt 1.

##### Beurt 1

In beurt 1 werd een gele knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />de kans dat je begint aan tafel ❶ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{3}{12} = \frac{1}{4}$, dus de kans dat je via tafel ❶ een gele knikker trekt is

$$
P(❶_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{4} = \frac{1}{12}
$$

* Vanuit tafel ❷:<br />de kans dat je begint aan tafel ❷ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{6}{12} = \frac{1}{2}$, dus de kans dat je via tafel ❷ een gele knikker trekt is

$$
P(❷_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{2} = \frac{1}{6}
$$

* Vanuit tafel ❸:<br />de kans dat je begint aan tafel ❸ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{0}{12} = 0$, dus de kans dat je via tafel ❸ een gele knikker trekt is

$$
P(❸_1 \cap \text{geel}_1) = \frac{1}{3} \cdot 0 = 0
$$

We kunnen dit samenvatten in een tabel:

| **Beurt:** | 1     |
| ---------: | :---: |
|            | p = $\frac{1}{12}$ via ❶ |
|            | p = $\frac{1}{6}$ via ❷ |
|            | p = $0$ via ❸ |

Het meest waarschijnlijk, tot dusverre, is dat de toestand behorende bij beurt 1 overeenkomt met tafel ❷, want dat heeft de hoogste kans.

##### Beurt 2

In beurt 2 werd een groene knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />Je kan op drie manieren aan tafel ❶ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{864} \approx 0.00116$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{432} \approx 0.00231$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{4}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{4}{6} \cdot \frac{1}{12} = 0 = 0.00000$

De *meest waarschijnlijke* manier om in beurt 2 aan tafel ❶ te komen is dus via tafel ❷ in beurt 1, met kans

$$
P(❷_1 \cap \text{geel}_1 \cap ❶_2 \cap \text{groen}_2) = 0.00231
$$

* Vanuit tafel ❷:<br />Je kan op drie manieren aan tafel ❷ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{3}{6} \cdot \frac{2}{12} = \frac{1}{144} \approx 0.00694$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{2}{6} \cdot \frac{2}{12} = \frac{1}{108} \approx 0.00926$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{2}{12} = 0 = 0.00000$

De *meest waarschijnlijke* manier om in beurt 2 aan tafel ❷ te komen is dus via tafel ❷ in beurt 1, met kans

$$
P(❷_1 \cap \text{geel}_1 \cap ❷_2 \cap \text{groen}_2) = 0.00926
$$

* Vanuit tafel ❸:<br />Je kan op drie manieren aan tafel ❸ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{2}{6} \cdot \frac{6}{12} = \frac{1}{72} \approx 0.01389$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{3}{6} \cdot \frac{6}{12} = \frac{1}{24} \approx 0.04167$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{6}{12} = 0 = 0.00000$

De *meest waarschijnlijke* manier om in beurt 2 aan tafel ❸ te komen is dus via tafel ❷ in beurt 1, met kans

$$
P(❷_1 \cap \text{geel}_1 \cap ❸_2 \cap \text{groen}_2) = 0.04167
$$

Als we nu alles tezamen bekijken, is de meest waarschijnlijke toestand in beurt 2 tafel ❸ geweest, volgend op tafel ❷ in beurt 1. Dat gaf immers een kans van 0.04167, wat hoger is dan beide andere.

We breiden hiermee de tabel uit:

| **Beurt:** | 1     | 2     |
| ---------: | :---: | :---: |
|            | p = $\frac{1}{12}$ via ❶ | p = 0.00231 via ❷-❶ |
|            | p = $\frac{1}{6}$ via ❷ | p = 0.00926 via ❷-❷ |
|            | p = $0$ via ❸ | p = 0.04167 via ❷-❸ |

##### Enzovoorts

We voeren steeds hetzelfde recept uit. Voor elke mogelijke tafel in de huidige beurt, beschouw elke mogelijke tafel uit de vorige beurt. Vermenigvuldig de kans die je uit die vorige beurt voor die tafel berekend had met de kans om van de vorige naar de huidige tafel over te gaan, en met de kans om dan de huidige kleur knikker te trekken. De meeste waarschijnlijke mogelijkheid "wint".

Ga na dat je de volgende resultaten krijgt:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
|            | p = $\frac{1}{12}$ via ❶ | p = 0.00231 via ❷-❶ | p = 0.01389 via ❷-❸-❶ | p = 0.00039 via ❷-❸-❶-❶ | p = 0.00011 via ❷-❸-❶-❸-❶ |
|            | p = $\frac{1}{6}$ via ❷ | p = 0.00926 via ❷-❷ | p = 0.00116 via ❷-❸-❷ | p = 0.00116 via ❷-❸-❶-❷ | p = 0.00006 via ❷-❸-❶-❷-❷ |
|            | p = $0$ via ❸ | p = 0.04167 via ❷-❸ | p = 0.00058 via ❷-❸-❸ | p = 0.00193 via ❷-❸-❶-❸ | p = 0.00029 via ❷-❸-❶-❷-❸ |

De eindconclusie lezen we nu af in de laatste kolom: de emissies geel-groen-blauw-rood-groen zijn het meest waarschijnlijk gekomen vanuit toestanden ❷-❸-❶-❷-❸!

Merk op dat er iets bijzonders kan gebeuren: in beurt vier was de meest waarschijnlijke reeks toestanden ❷-❸-❶-❸ (met een kans 0.00193), en leek het er dus op dat je in beurt vier aan tafel ❸ zat. Echter, uiteindelijk bleek de meest waarschijnlijke reeks ❷-❸-❶-❷-❸ (met kans 0.00029), en zat je in beurt vier dus aan tafel ❷. Dat wil zeggen, latere informatie heeft invloed op de beste oplossing voor eerdere beurten.

De vorige les berekenden we de kans dat je exact dezelfde reeks "geel, groen, blauw, rood, groen" zou vinden als je achtereenvolgens de tafels ❷-❸-❶-❸-❷ zou bezoeken. We vonden toen een kans $p = 0.000054$. Nu zien we hierboven dat het VIterbi algoritme een *andere* reeks tafels produceert: ❷-❸-❶-❷-❸. Hiervoor vinden we echter een kans $p = 0.00029$. Inderdaad is die kans hoger, dus is het waarschijnlijker dat de observaties afkomstig zijn van deze reeks als je niet weet van welke tafels ze daadwerkelijk komen. Merk ook op dat de meest waarschijnlijke reeks tafels toch behoorlijk wat overlap heeft met de werkelijke reeks tafels die we niet kenden. Het Viterbi algoritme produceert dus best een redelijke schatting omtrent wat de verborgen toestanden geweest zouden zijn.

Bepaal nu voor de eerste vijf beurten uit jouw eigen experimentele waarnemingen wat de meest waarschijnlijke bijbehorende serie tafels is. Is dit dezelfde reeks als de tafels die je daadwerkelijk bezocht hebt?

|   Jasper    | 1                                                                                      | 2                                                                                  | 3                                                                                            | Meest waarchijnlijk  |
|:-----:|----------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------|----------------------|
|  geel | p=1/3*1/4= 1/12                                                                        | p=1/3*1/2= 1/6                                                                     | p=1/3*0= 0                                                                                   | tafel 2              |
| groen | p= 1/12 *1/6 * 1/12= 1/864  p= 1/6 *1/6 * 1/12 = 1/432 ! p= 0 *2/3 * 1/12= 0           | p= 1/12 *1/2 * 1/6 1/144 p= 1/6 *1/3 * 1/6 = ! p = 0 * 1/6 *1/6 = 0                | p= 1/12 *1/3 * 1/2 1/72 p= 1/6 *1/2 * 1/2 = 1/24! p = 0 * 1/6 *1/2 = 0                       | tafel 2-3            |
|  geel | p= 1/432*1/6 * 1/4 = 1/10368 p= 1/108 *1/6 * 1/4 = 1/2592 p = 1/24* 2/3 *1/4 = 1/144 ! | p= 1/432*1/2 * 1/2= 1/1728 p= 1/108 *1/3 * 1/2 = 1/648 p = 1/24* 1/6 *1/2 = 1/288! | p = 0 p=0 p=0                                                                                | Tafel 231            |
| groen | p= 1/144*1/6*1/12= 1/10368! p= 1/288*1/6*1/12= 1/20736 p= 0                            | p= 1/288*1/2*1/6= 1/3456! p= 1/288*1/3*1/6= 1=5184 p= 0                            | p= 1/288*1/3*1/2= 1/1728 p= 1/288*1/2*1/2= 1/1152! p= 0                                      | Tafel 2313           |
| blauw | p= 20736*1/6*1/2 = 1/248832 p= 3456*1/6*1/2= 1/41472 p= 1/1152*2/3*1/2= 1/3456!        | p= 1/20736*1/2*1/6=248832! p= 1/3456 *1/3*1/6= 62208 p= 1/1152* 1/6 *1/6= 41472    | p = 1/20736 *1/3*1/12=1/746496 p = 1/3456  *1/2*1/12=1/82944! p = 1/1152  *1/6*1/12=1/82944! | Tafel 23131          |



|     Ramon  | 1                                                                             | 2                                                                             | 3                                                              | Meest waarchijnlijk  |
|:-----:|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------|----------------------------------------------------------------|----------------------|
| groen | p= 1/3 *1/12= 1/36                                                            | p=1/3 *1/6= 1/18                                                              | p=1/3*1/2=1/6                                                  | 3                    |
|  geel | p= 1/36*1/6*1/4=1/864 p=1/18*1/6*1/4=1/432 p=1/6*2/3*1/4=1/36!                | p= 1/36*1/2*1/2=1/144 p=1/18*1/3*1/2=1/108 p=1/6*1/6*1/2=1/72!                | p= 1/36*1/3*0 p=1/18*1/2*0 p=1/6*1/6*0                         | 31                   |
|  rood | p= 1/36*1/6*1/1296! p=1/72*1/6*1/2592 p=0*1/6*1/6=0                           | p= 1/36*1/2*1/6=1/432! p=1/72*1/3*1/6=1/1296 p=0*1/6*1/6= 0                   | p= 1/36*1/3*5/12=5/1296! p=1/72*1/2*5/12=5/1728 p=0*1/6*5/12=0 | 312                  |
|  geel | p = 1/1296*1/6*1/4=1/31104 p= 1/432*1/6*1/4=1/10368 p= 5/1296*2/3*1/4=5/7776! | p= 1/1296*1/2*1/2=1/5184 p= 1/432*1/3/*1/2= 1/2592! p= 5/1296*1/6*1/2=5/15552 | p=0  p=0  p=0                                                  | 3122                 |
| groen | p= 5/7776 * 1/6*1/12=5/559872 p= 1/2592*1/6*1/12=1/186624! p=0                | p= 5/7776*1/2*1/6=5/93312! p= 1/2592*1/3*1/6=1/46656 p=0                      | p=5/7776*1/3*1/2=5/46656! p=1/2592*1/2*1/2=1/10368 p=0         | 31223                |


<a id="3" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Rekenvoorbeeld

De onderstaande video behandelt het decoding-probleem voor een Hidden Markov Model met twee toestanden: warm of koud weer; en drie mogelijke emissies: één, twee of drie gegeten ijsjes. (Dit voorbeeld wordt desgewenst [hier](https://www.youtube.com/watch?v=xejm-z3sbWA) nader toegelicht.) In de video wordt uitgewerkt hoe je kan nagaan wat de meest waarschijnlijke serie warme of koude dagen is geweest, gegeven dat je een aantal dagen observeert hoeveel ijsjes er gegeten worden.

Bekijk de video en ga na dat je begrijpt hoe het Viterbi-algoritme in dit geval wordt toegepast om het decoding-probleem op te lossen. Als je het algoritme reeds goed denkt te begrijpen, bekijk dan de video vluchtig; als je het een ingewikkeld algoritme vindt, probeer dan daadwerkelijk elke rekenstap zelf na te doen. Als je zaken opmerkt die je noemenswaardig vindt om te onthouden, noteer die dan.

Opmerking: de audio-kwaliteit en verstaanbaarheid van deze video is niet optimaal; het voorbeeld zelf is echter zeer illustratief en geschikt om door te nemen.

In [ ]:
%%html
<iframe width="640" height="360" src="https://www.youtube.com/embed/xejm-z3sbWA" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [ ]:
# UITWERKING

<a id="4" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Je eigen `HiddenMarkovModel` class

Voeg nu een `predict()` methode toe aan je eigen `HiddenMarkovModel` klasse. Deze ontvangt slechts één argument: een iterable met emissies (hier de reeks kleuren; bv. `X`). De methode dient op grond van de bekende begintoestandverdeling, emissiekansen, en overgangswaarschijnlijkheden te bepalen wat de meest waarschijnlijke reeks toestanden is die overeenkomt met de gegeven emissies. Als retourwaarde wordt een iterable teruggegeven aan de gebruiker met toestanden (hier de reeks tafels; `state_sequence`). Zie de [documentatie](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.CategoricalHMM.predict) van `hmmlearn` voor een voorbeeld van een soortgelijke API.

Opnieuw geldt dat er een probleem kan ontstaan als je vele opeenvolgende waarnemingen moet doorrekenen: de kansen worden geleidelijk aan steeds kleiner, en kunnen op den duur mogelijk niet meer door een floating-point waarde gerepresenteerd worden omdat ze afgerond worden naar nul. Bij het decoding-probleem gaat het er slechts om om de toestanden met de hoogste kans te achterhalen; het berekenen van die kans deden we in het vorige deel. Daarom kun je dit numerieke afrond-probleem oplossen door alle kansen samen te schalen. Ter illustratie, in beurt 5 vonden we hierboven kansen p = 0.00011, 0.00006, en 0.00029; je zou dit kunnen opschalen naar bijvoorbeeld p = 0.11, 0.06, en 0.29. De optimale reeks toestanden verandert daardoor niet.

Gebruik je eigen module om de berekening uit de voorgaande oefening voor je eerste vijf waarnemingen te controleren. Komt er hetzelfde antwoord uit? Bepaal ook de meest waarschijnlijke reeks tafels voor jouw hele eigen reeks waarnemingen. Hoeveel procent van de toestanden wordt goed voorspeld? Is dit meer dan een fractie $\frac{1}{3}$ die je op grond van toeval zou verwachten?

Gebruik de `score()` functie van je eigen module om de log-waarschijnlijkheid op de waargenomen kleuren te bepalen: vergelijk de waarschijnlijkheid op basis van de zojuist bepaalde meest waarschijnlijke reeks tafels met die op basis van de daadwerkelijke reeks tafels die je bezocht hebt. Verifieer dat de kans met de meest waarschijnlijke reeks inderdaad de hoogste is. Het kan dus zo zijn dat een andere reeks toestanden dan die welke tot de emissies leidden beter overeenkomen met de emissies!

Doe hetzelfde met de waarnemingen van de klas als geheel. Hoeveel procent van de toestanden kan je Hidden Markov Model juist voorspellen? Komt dit antwoord overeen met wat `CategoricalHMM(...).predict(X)` uit de `hmmlearn` module rapporteert?

In [1]:
from hmmmodel import HiddenMarkovModel
import numpy as np
states = np.array([0,1,2,0,2])
emissions = np.array([2, 1, 3, 1, 2])

own_model = HiddenMarkovModel(3, 4)
own_model.startprob_ = np.array([1/3, 1/3, 1/3])
own_model.transmat_ = np.array([
    [1/6, 1/2, 1/3],
    [1/6, 1/3, 1/2],
    [4/6, 1/6, 1/6]
])
# blauw, geel, groen rood
own_model.emissionprob_ = np.array([
    [1/2, 1/4, 1/12, 1/6],
    [1/6, 1/2, 1/6, 1/6],
    [1/12, 0, 1/2, 5/12]
])

In [2]:
emissions = np.array([3, 0, 1, 1, 2])
own_model.predict(emissions)

array([2, 0, 1, 1, 2])

In [3]:
from hmmlearn.hmm import CategoricalHMM as HMM

learn_model = HMM(3,4)
learn_model.startprob_ = np.array([1/3, 1/3, 1/3])
learn_model.transmat_ = np.array([
    [1/6, 1/2, 1/3],
    [1/6, 1/3, 1/2],
    [4/6, 1/6, 1/6]
])
# blauw, geel, groen rood
learn_model.emissionprob_ = np.array([
    [1/2, 1/4, 1/12, 1/6],
    [1/6, 1/2, 1/6, 1/6],
    [1/12, 0, 1/2, 5/12]
])


In [4]:
learn_model.predict(emissions.reshape(-1,1))

array([2, 0, 1, 1, 2])

De predictie van ons model en het learnhmmmodel zijn in deze situatie gelijk. Nu ga ik kijken of ons model dezelfde staten kan voorspellen op basis van de 1e 5 datapunten uit jasper en mijn data


In [5]:
ramons_emissie = np.array([2, 1, 3, 1, 2])
ramons_states = np.array([0,1,2,0,2])
jaspers_emissie = np.array([1,2,1,2,0])
jaspers_states = np.array([1,2,0,2,0])

In [7]:
def check_predicted_states(true_states, predicted_states):
    print(f"Predicted states: {predicted_states}")
    amount_equal = np.sum(predicted_states == true_states)
    print(f"{amount_equal} states are equal")
    compare = "larger" if (amount_equal / true_states.size) > (1/3) else "smaller"

    print(f"This is {compare} then the 1/3 fraction of coincidence")

In [8]:
predicted_states = own_model.predict(ramons_emissie)
check_predicted_states(ramons_states, predicted_states)


Predicted states: [2 0 2 0 2]
3 states are equal
This is larger then the 1/3 fraction of coincidence


In [9]:
predicted_states_jasper = own_model.predict(jaspers_emissie)
check_predicted_states(jaspers_states, predicted_states_jasper)

Predicted states: [1 2 0 2 0]
5 states are equal
This is larger then the 1/3 fraction of coincidence


## data hele klas


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("./dataklass.csv")
data.head()

In [ ]:
emissions_class = data.get("#emission")
states_class = data.get("#state")

In [ ]:
predicted_states_class_own = own_model.predict(emissions_class.to_numpy())

In [ ]:
n_equal_own = np.sum(states_class == predicted_states_class_own)
print(f"{n_equal_own} predicated and true states were equal of the {states_class.size}. {n_equal_own / states_class.size * 100}%")

In [ ]:
predicted_states_class = learn_model.predict(emissions_class.to_numpy().reshape(-1,1))

In [ ]:
n_equal = np.sum(states_class == predicted_states_class)
print(f"{n_equal} predicated and true states were equal of the {states_class.size}. {n_equal / states_class.size * 100}%")

In [ ]:
# UITWERKING

<a id="5" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-eilandjes

In deel 1 genereerde je random sequenties op basis van transitiewaarschijnlijkheden zoals die voorkomen binnen en buiten CpG-eilandjes. Als we kijken naar de frequentie van verschillende nucleotiden in deze twee verschillende regio's, dan vinden we getallen zoals ongeveer hieronder:

| Emissies          |   A   |   C   |   G   |   T   |
| :---------------: | :---: | :---: | :---: | :---: |
| **CGI** (`+`)     | 0.20  | 0.30  | 0.30  | 0.20  |
| **non-CGI** (`-`) | 0.25  | 0.25  | 0.25  | 0.25  |

CpG-eilandjes zijn gemiddeld enkele honderden base-paren lang, en worden onderling gescheiden door typisch enkele honderden tot duizenden base-paren. We kunnen daarom bijvoorbeeld een transitiematrix schatten als volgt:

| Transities        | CGI (`+`)     | non-CGI (`-`) |
| :---------------: | :-----------: | :-----------: |
| **CGI** (`+`)     |    0.990      |    0.010      |
| **non-CGI** (`-`) |    0.001      |    0.999      |

Definieer een Hidden Markov Model op basis van bovenstaande gegevens en gebruik dit om de sequentie in het meegeleverde bestand `Casus_HiddenMarkovModel.fasta` te analyseren. Kies zelf geschikte startwaarschijnlijkheden. Op welke plekken vind je CpG-eilandjes? Wat is het GC-gehalte van alle CGI regios tezamen, en wat is het GC-gehalte van alle non-CGI regios?

Tenslotte, vergelijk je eigen resultaten met die van een van onderstaande online tools:

* https://www.bioinformatics.org/sms/cpg_island.html

* https://groups.molbiosci.northwestern.edu/matouschek/links/sms2/cpg_islands.html

* https://www.genscript.com/sms2/cpg_islands.html

Deze maken geen gebruik van Hidden Markov Modellen, maar gebruiken een sliding-window techniek. Kun je iets zeggen over de mate waarin je eigen resultaten overeenkomen met die van dit tool? Kun je speculeren wat een reden kan zijn voor de verschillen?

In [10]:
import numpy as np

hmm = HiddenMarkovModel(n_components=2, n_features=4)

# startwaarschijnlijkheden (CGI, non-CGI)
hmm.startprob_ = np.array([0.5, 0.5])


# transitie-matrix
hmm.transmat_ = np.array([
    [0.990, 0.010],  # van CGI (+) naar (CGI, non-CGI)
    [0.001, 0.999],  # van non-CGI (-) naar (CGI, non-CGI)
])

# emissie-matrix: rijen = states, kolommen = A,C,G,T
hmm.emissionprob_ = np.array([
    [0.20, 0.30, 0.30, 0.20],  # CGI
    [0.25, 0.25, 0.25, 0.25],  # non-CGI
])




In [11]:
def read_fasta_as_indices(path):
    nuc2idx = {"A": 0, "C": 1, "G": 2, "T": 3}

    seq = []
    with open(path) as f:
        for line in f:
            line = line.strip().upper()
            if not line or line.startswith(">"):
                continue
            for ch in line:
                if ch in nuc2idx:         # eventuele N's negeren of apart behandelen
                    seq.append(nuc2idx[ch])
                # anders: skippen of error, afhankelijk van wat je wilt.
    return np.array(seq, dtype=int)

X = read_fasta_as_indices("../Casus_HiddenMarkovModel.fasta")



In [12]:
states = hmm.predict(X)   # array met 0 (CGI) of 1 (non-CGI) per positie


In [13]:
def get_segments(states, state_of_interest=0, min_len=1):
    segments = []
    n = len(states)
    in_seg = False
    start = None

    for i in range(n):
        if states[i] == state_of_interest:
            if not in_seg:
                in_seg = True
                start = i
        else:
            if in_seg:
                end = i - 1
                if end - start + 1 >= min_len:
                    segments.append((start, end))
                in_seg = False
    # eventueel lopend segment afmaken
    if in_seg:
        end = n - 1
        if end - start + 1 >= min_len:
            segments.append((start, end))
    return segments

# bijv. alleen eilanden van minimaal 200 bp
cpg_segments = get_segments(states, state_of_interest=0, min_len=200)

# 1-based posities om in je verslag te zetten
cpg_segments_1based = [(s+1, e+1) for (s, e) in cpg_segments]
for i, (s, e) in enumerate(cpg_segments_1based, start=1):
    print(f"CpG-eiland {i}: van positie {s} tot {e} (lengte {e-s+1} bp)")

CpG-eiland 1: van positie 9612 tot 10876 (lengte 1265 bp)
CpG-eiland 2: van positie 25682 tot 26336 (lengte 655 bp)
CpG-eiland 3: van positie 35994 tot 36967 (lengte 974 bp)
CpG-eiland 4: van positie 50295 tot 50655 (lengte 361 bp)
CpG-eiland 5: van positie 58769 tot 59384 (lengte 616 bp)
CpG-eiland 6: van positie 71691 tot 72000 (lengte 310 bp)


According to sliding window (www.bioinformatics.org)
* Found  295-bp CpG-island at position  1306 to  1600
* Found  318-bp CpG-island at position  4826 to  5143
* Found  208-bp CpG-island at position  8850 to  9057
* Found 1565-bp CpG-island at position  9489 to 11053
* Found  326-bp CpG-island at position 13425 to 13750
* Found  212-bp CpG-island at position 14567 to 14778
* Found  246-bp CpG-island at position 21647 to 21892
* Found  804-bp CpG-island at position 25521 to 26324
* Found  200-bp CpG-island at position 27171 to 27370
* Found  309-bp CpG-island at position 28247 to 28555
* Found  308-bp CpG-island at position 34682 to 34989
* Found 1529-bp CpG-island at position 35724 to 37252
* Found  325-bp CpG-island at position 39314 to 39638
* Found  411-bp CpG-island at position 47078 to 47488
* Found  214-bp CpG-island at position 48130 to 48343
* Found  259-bp CpG-island at position 48512 to 48770
* Found  213-bp CpG-island at position 49263 to 49475
* Found  470-bp CpG-island at position 50254 to 50723
* Found  720-bp CpG-island at position 58688 to 59407
* Found  339-bp CpG-island at position 60759 to 61097
* Found  234-bp CpG-island at position 61631 to 61864
* Found  212-bp CpG-island at position 61891 to 62102
* Found  226-bp CpG-island at position 62672 to 62897
* Found  296-bp CpG-island at position 63541 to 63836
* Found  201-bp CpG-island at position 63950 to 64150
* Found  201-bp CpG-island at position 64835 to 65035
* Found  349-bp CpG-island at position 67031 to 67379
* Found  220-bp CpG-island at position 71411 to 71630
* Found  346-bp CpG-island at position 71654 to 72000

In [14]:
X = np.array(X)
states = np.array(states)

# indices van CGI- en non-CGI-posities
cgi_idx = np.where(states == 0)[0]
non_idx = np.where(states == 1)[0]

def gc_content(indices):
    if len(indices) == 0:
        return float("nan")
    subseq = X[indices]
    gc = np.sum((subseq == 1) | (subseq == 2))  # 1 = C, 2 = G
    return gc / len(indices)

gc_cgi = gc_content(cgi_idx)
gc_non = gc_content(non_idx)

print("GC-gehalte CGI-regio's   :", gc_cgi)
print("GC-gehalte non-CGI-regio's:", gc_non)


GC-gehalte CGI-regio's   : 0.6893087778043531
GC-gehalte non-CGI-regio's: 0.48493784927527683


In deze analyse is gebruikgemaakt van de online tool “CpG Islands” van [Bioinformatics.org](https://www.bioinformatics.org/sms/cpg_island.html), die werkt volgens de methode van Gardiner‑Garden & Frommer (1987). Deze tool gebruikt een 200 bp window dat elke 1 bp verschuift, en markeert regio’s als CpG-eiland indien zowel het GC-gehalte > 50% is als de CpG observed/expected-waarde (y-waarde) > 0.6.

In vergelijking met onze HMM-aanpak vindt deze sliding-window tool meer en vaak kortere CpG-eilanden. Dit komt doordat de tool in elk window los beslist op basis van vaste drempels, terwijl het HMM structureel de voorkeur geeft aan non-CGI-toestanden en overgangen beheerst via probabilistische parameters.

***

&copy; 2025 - Dave R.M. Langers <d.r.m.langers@pl.hanze.nl>